In [1]:
!pip install langchain_huggingface
!pip install langchain_community
!pip install langgraph
!pip install sentence_transformers
!pip install transformers
!pip install chromadb pysqlite3-binary

In [2]:
from google.colab import drive
drive.mount('/content/drive')

# 3. VERIFY your path exists (Debug Step)
import os
MY_PATH = "/content/drive/MyDrive/llmFinal/qwen2.5-cai-dpo-final"

if os.path.exists(MY_PATH):
    print(f"Success! Found your model at: {MY_PATH}")
else:
    print(f"Error! Could not find path: {MY_PATH}")
    print("Please check your Google Drive folder structure.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Error! Could not find path: /content/drive/MyDrive/llmFinal/qwen2.5-cai-dpo-final
Please check your Google Drive folder structure.


In [3]:
!pip install -qU langchain-text-splitters langchain-core

In [8]:
import os
import torch
import pandas as pd
from typing import List, TypedDict
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
from peft import PeftModel
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import END, StateGraph
from sentence_transformers import SentenceTransformer, util

# ==========================================
# 1. CONFIGURATION
# ==========================================
BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"

FINETUNED_PATH = "/content/drive/MyDrive/llmFinal/qwen2.5-cai-dpo-final"

DATASET_PATHS = [
    "/content/drive/MyDrive/llmFinal/datasets/airport_ticket.csv",
    "/content/drive/MyDrive/llmFinal/datasets/fastfood.csv",
    "/content/drive/MyDrive/llmFinal/datasets/finance.csv",
    "/content/drive/MyDrive/llmFinal/datasets/media.csv",
    "/content/drive/MyDrive/llmFinal/datasets/software.csv",
]

TAVILY_API_KEY = "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"
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

# ==========================================
# 2. LOAD MODELS
# ==========================================
print("Loading models...")

base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

if os.path.exists(FINETUNED_PATH):
    try:
        from peft import PeftConfig

        # Try to load with compatibility settings
        config = PeftConfig.from_pretrained(FINETUNED_PATH)
        model = PeftModel.from_pretrained(
            base_model,
            FINETUNED_PATH,
            is_trainable=False,
            low_cpu_mem_usage=True
        )
        print("✅ Fine-tuned adapters loaded!")
    except Exception as e:
        print(f"Failed to load adapters: {e}")
        print("Using base model only.")
        model = base_model
else:
    print("Path not found. Loading base model only.")
    model = base_model

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,
    temperature=0.7,
    repetition_penalty=1.2,
    return_full_text=False
)

generator_llm = HuggingFacePipeline(pipeline=pipe)
similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# ==========================================
# 3. BUILD KNOWLEDGE BASE
# ==========================================
print("Processing CSVs...")

def load_conversations(csv_paths, max_per_dataset=200):
    all_docs = []
    for csv_path in csv_paths:
        if not os.path.exists(csv_path):
            continue
        try:
            df = pd.read_csv(csv_path)
            if 'utterance' not in df.columns: continue

            df = df.dropna(subset=['utterance'])
            df['utterance'] = df['utterance'].astype(str).str.strip()

            count = 0
            for conv_id, group in df.groupby('conversationId'):
                if count >= max_per_dataset: break

                text = ""
                for _, row in group.iterrows():
                    role = "Customer" if row.get('authorRole') == 'customer' else "Agent"
                    text += f"{role}: {row['utterance']}\n"

                all_docs.append(Document(page_content=text, metadata={"source": csv_path}))
                count += 1
            print(f"Loaded {count} convs from {os.path.basename(csv_path)}")
        except:
            continue
    return all_docs

docs = load_conversations(DATASET_PATHS)

if not docs:
    print("No documents found! Check your Drive paths.")
else:
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
    split_docs = text_splitter.split_documents(docs)

    vectorstore = Chroma.from_documents(
        documents=split_docs,
        collection_name="colab_kb_fixed",
        embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
    print("Knowledge Base Ready!")

# ==========================================
# 4. DEFINE AGENT
# ==========================================

class GraphState(TypedDict):
    question: str
    generation: str
    web_search: str
    documents: List[str]
    urgency: str      # INPUT from external agent
    emotion: str      # INPUT from external agent

def retrieve(state):
    """Retrieve relevant documents"""
    print(f"\n--- RETRIEVING: {state['question']} ---")
    docs = retriever.invoke(state["question"])
    return {**state, "documents": docs}

def grade_documents(state):
    """Grade document relevance and decide if web search is needed"""
    question = state["question"]
    documents = state["documents"]

    # Check for real-time queries
    if any(x in question.lower() for x in ["weather", "today", "now", "price", "stock"]):
        print("  Real-time query -> WEB SEARCH")
        return {**state, "web_search": "Yes"}

    # No documents found
    if not documents:
        return {**state, "documents": [], "web_search": "Yes"}

    # Calculate similarity
    q_emb = similarity_model.encode(question, convert_to_tensor=True)
    max_sim = 0
    for doc in documents:
        d_emb = similarity_model.encode(doc.page_content[:400], convert_to_tensor=True)
        max_sim = max(max_sim, util.cos_sim(q_emb, d_emb).item())

    print(f" Similarity Score: {max_sim:.2f}")

    if max_sim < 0.4:
        print("  Low Relevance -> WEB SEARCH")
        return {**state, "web_search": "Yes"}

    print("  High Relevance -> GENERATE")
    return {**state, "web_search": "No"}

def web_search(state):
    """Perform web search for additional context"""
    print("--- WEB SEARCH ---")
    try:
        results = TavilySearchResults(k=3).invoke({"query": state["question"]})
        web_text = "\n".join([r["content"] for r in results])
        documents = state["documents"] + [Document(page_content=web_text)]
        print("  Web results found")
        return {**state, "documents": documents}
    except Exception as e:
        print(f"Web search failed: {e}")
        return state

def generate(state):
    """Generate response based on context, urgency, and emotion"""
    print("--- GENERATING ---")
    question = state["question"]
    documents = state["documents"]
    urgency = state.get("urgency", "medium").lower()
    emotion = state.get("emotion", "neutral").lower()

    context = "\n\n".join([d.page_content[:500] for d in documents])
    print(f"CONTEXT SEEN BY MODEL:\n{context[:200]}...\n(Total chars: {len(context)})")
    print(f"Urgency: {urgency.upper()} | Emotion: {emotion.upper()}")

    # ========== EXACT TONE MATCHING FOR ALL SCENARIOS ==========

    # HIGH URGENCY
    if urgency == "high" and emotion == "angry":
        tone_instruction = "Be extremely empathetic and apologetic. Acknowledge the issue immediately and provide concrete solutions with urgency."
    elif urgency == "high" and emotion == "frustrated":
        tone_instruction = "Be understanding and solution-focused. Act quickly to resolve the issue and show you're prioritizing their concern."
    elif urgency == "high" and emotion == "anxious":
        tone_instruction = "Be reassuring and calm while acting swiftly. Provide clear steps and timeline to reduce their anxiety."
    elif urgency == "high" and emotion == "neutral":
        tone_instruction = "Be direct, efficient, and action-oriented. Provide immediate solutions without unnecessary details."
    elif urgency == "high" and emotion == "satisfied":
        tone_instruction = "Be appreciative and maintain the positive experience. Act quickly while being warm and professional."
    elif urgency == "high" and emotion == "happy":
        tone_instruction = "Be enthusiastic and efficient. Match their positive energy while resolving their urgent request quickly."

    # MEDIUM URGENCY
    elif urgency == "medium" and emotion == "angry":
        tone_instruction = "Be patient and understanding. Acknowledge their frustration and work through the solution methodically."
    elif urgency == "medium" and emotion == "frustrated":
        tone_instruction = "Be empathetic and helpful. Show you understand their concern and provide clear, actionable solutions."
    elif urgency == "medium" and emotion == "anxious":
        tone_instruction = "Be supportive and reassuring. Provide clear guidance and communicate each step to ease their worry."
    elif urgency == "medium" and emotion == "neutral":
        tone_instruction = "Be friendly, helpful, and professional. Provide thorough assistance with a balanced approach."
    elif urgency == "medium" and emotion == "satisfied":
        tone_instruction = "Be warm and attentive. Maintain their satisfaction while helping them with their request."
    elif urgency == "medium" and emotion == "happy":
        tone_instruction = "Be friendly and engaging. Match their positive mood while being helpful and thorough."

    # LOW URGENCY
    elif urgency == "low" and emotion == "angry":
        tone_instruction = "Be calm, patient, and thorough. Take time to fully explain and address their concerns."
    elif urgency == "low" and emotion == "frustrated":
        tone_instruction = "Be understanding and patient. Provide detailed explanations and show genuine care for resolving their issue."
    elif urgency == "low" and emotion == "anxious":
        tone_instruction = "Be gentle, reassuring, and thorough. Take time to address their concerns and provide comprehensive support."
    elif urgency == "low" and emotion == "neutral":
        tone_instruction = "Be professional, informative, and helpful. Provide clear and complete information."
    elif urgency == "low" and emotion == "satisfied":
        tone_instruction = "Be friendly and informative. Maintain the positive relationship while helping them effectively."
    elif urgency == "low" and emotion == "happy":
        tone_instruction = "Be warm, friendly, and conversational. Enjoy the positive interaction while being helpful."

    else:
        tone_instruction = "Be helpful and professional."

    # ========== END TONE MATCHING ==========

    rag_system_message = f"""CRITICAL INSTRUCTIONS - FOLLOW EXACTLY:
1. You are a GENERAL helpful assistant, NOT a ticket agent
2. NEVER mention company names, websites, or booking services
3. Customer Urgency: {urgency.upper()}
4. Customer Emotion: {emotion.upper()}
5. Tone: {tone_instruction}

ANSWER THE QUESTION using ONLY the Context provided below.
If the Context has the answer, use it.
If the Context doesn't have the answer, say "I don't have that information, I will do online search."
NEVER make up information. NEVER mention companies or websites.
Maximum 2-3 sentences."""

    prompt = f"""<|im_start|>system
{rag_system_message}<|im_end|>
<|im_start|>user
Context:
{context}

Question: {question}<|im_end|>
<|im_start|>assistant
"""

    result = pipe(prompt, max_new_tokens=150, temperature=0.5)[0]['generated_text']

    if "<|im_start|>assistant" in result:
        result = result.split("<|im_start|>assistant")[-1].strip()

    return {**state, "generation": result}

def route(state):
    """Route to web search or generate based on document grading"""
    return "web_search" if state["web_search"] == "Yes" else "generate"

# ==========================================
# 5. BUILD WORKFLOW
# ==========================================

workflow = StateGraph(GraphState)

workflow.add_node("retrieve", retrieve)
workflow.add_node("grade_documents", grade_documents)
workflow.add_node("web_search", web_search)
workflow.add_node("generate", generate)

workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges("grade_documents", route, {"web_search": "web_search", "generate": "generate"})
workflow.add_edge("web_search", "generate")
workflow.add_edge("generate", END)

app = workflow.compile()

# ==========================================
# 6. TEST
# ==========================================
print("\n" + "="*50)
print("TEST 1: High Urgency + Angry")
inputs = {
    "question": "I want to book a flight to Chicago",
    "urgency": "high",
    "emotion": "angry"
}
for output in app.stream(inputs):
    pass
print(f"\n>>> ANSWER: {output['generate']['generation']}")

print("\n" + "="*50)
print("TEST 2: Medium Urgency + Neutral")
inputs = {
    "question": "What is the weather in Tokyo right now?",
    "urgency": "medium",
    "emotion": "neutral"
}
for output in app.stream(inputs):
    pass
print(f"\n>>> ANSWER: {output['generate']['generation']}")

Loading models...
Path not found. Loading base model only.


Device set to use cuda:0


Processing CSVs...
Loaded 200 convs from airport_ticket.csv
Loaded 200 convs from fastfood.csv
Loaded 200 convs from finance.csv
Loaded 200 convs from media.csv
Loaded 200 convs from software.csv
Knowledge Base Ready!

TEST 1: High Urgency + Angry

--- RETRIEVING: I want to book a flight to Chicago ---
 Similarity Score: 0.53
  High Relevance -> GENERATE
--- GENERATING ---
CONTEXT SEEN BY MODEL:
Customer: hi
Agent: Good morning! You've reached Alliance Airways Customer service, how may I help you?
Customer: "Book" a plane ticket
Agent: Sure! I'll help you with that, could you please let me kn...
(Total chars: 1504)
Urgency: HIGH | Emotion: ANGRY

>>> ANSWER: Apologies for any inconvenience caused. To assist you better, we need more details about your trip such as the specific dates of your journey from August 29th onwards. Once confirmed, our team will proceed with making arrangements for you in advance. Thank you for understanding.

TEST 2: Medium Urgency + Neutral

--- RETRIEVING: Wh

In [9]:
# ==========================================
# 6. TEST ALL 18 SCENARIOS
# ==========================================

test_cases = [
    # HIGH URGENCY
    ("medium", "anxious", "My flight leaves in 2 hours and I still haven't received my booking confirmation!"),
    ("high", "frustrated", "I've been trying to track my package for hours. I need this today!"),
    ("medium", "anxious", "I'm really worried about my order. It was supposed to arrive yesterday."),
    ("high", "anxious", "I need to change my reservation immediately. What are my options?"),
    ("high", "neutral", "Thank you for the service! I need to add one more item urgently."),
    ("medium", "happy", "This is amazing! Can you help me order three more right away?"),

    # MEDIUM URGENCY
    ("medium", "frustrated", "I'm not happy with the service. The product description was misleading."),
    ("medium", "frustrated", "I've contacted support twice about this. Can someone help?"),
    ("medium", "anxious", "I'm concerned about the payment process. Is my information secure?"),
    ("medium", "neutral", "Can you explain how the refund policy works?"),
    ("low", "happy", "I'm happy with my purchase. Tell me about your loyalty program."),
    ("low", "happy", "I'm enjoying my experience! What other products do you recommend?"),

    # LOW URGENCY
    ("medium", "frustrated", "I'm disappointed with this. I want a full explanation of what went wrong."),
    ("medium", "anxious", "This is confusing. Why is the process so complicated?"),
    ("medium", "anxious", "I'm not sure if I made the right choice. Can you explain the differences?"),
    ("medium", "neutral", "What are the technical specifications of this product?"),
    ("low", "happy", "Everything's been great. I'm curious about your sustainability practices."),
    ("low", "happy", "I love your products! Tell me about the story behind your company.")
]

print("\n" + "="*70)
print("TESTING 18 SCENARIOS: 3 URGENCY × 6 EMOTIONS")
print("="*70)

success_count = 0
web_search_count = 0

for i, (urgency, emotion, query) in enumerate(test_cases, 1):
    print(f"\n{'='*70}")
    print(f"TEST {i}/18: {urgency.upper()} + {emotion.upper()}")
    print(f"{'='*70}")
    print(f"Query: {query}")
    print("-"*70)

    inputs = {
        "question": query,
        "urgency": urgency,
        "emotion": emotion
    }

    try:
        for output in app.stream(inputs):
            pass

        response = output['generate']['generation']
        web_search = output['generate'].get('web_search', 'No')

        print(f"Web Search: {web_search}")
        print(f"\n>>> RESPONSE:\n{response}\n")

        success_count += 1
        if web_search == "Yes":
            web_search_count += 1

    except Exception as e:
        print(f"❌ ERROR: {e}\n")

print("\n" + "="*70)
print("SUMMARY")
print("="*70)
print(f"Successful: {success_count}/18")
print(f"Web Search Triggered: {web_search_count}/18")
print("="*70)


TESTING 18 SCENARIOS: 3 URGENCY × 6 EMOTIONS

TEST 1/18: MEDIUM + ANXIOUS
Query: My flight leaves in 2 hours and I still haven't received my booking confirmation!
----------------------------------------------------------------------

--- RETRIEVING: My flight leaves in 2 hours and I still haven't received my booking confirmation! ---
 Similarity Score: 0.65
  High Relevance -> GENERATE
--- GENERATING ---
CONTEXT SEEN BY MODEL:
Agent: Oh! That's okay! I'll change that. However, As I can check here both the numbers given to me are incorrect, Your booking confirmation number should be 6 characters (mix of letters and numbers)....
(Total chars: 1228)
Urgency: MEDIUM | Emotion: ANXIOUS
Web Search: No

>>> RESPONSE:
The customer is experiencing anxiety because they believe there might be an issue with receiving their booking confirmation before departure time. The agent reassures them by confirming that the correct booking confirmation number consists of 6 characters mixed between letters 

In [6]:
# import os
# import torch
# import pandas as pd
# from typing import List, TypedDict
# from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
# from langchain_huggingface import HuggingFacePipeline, HuggingFaceEmbeddings
# from peft import PeftModel
# from langchain_community.vectorstores import Chroma
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# from langchain_core.documents import Document
# from langchain_community.tools.tavily_search import TavilySearchResults
# from langgraph.graph import END, StateGraph
# from sentence_transformers import SentenceTransformer, util

# # ==========================================
# # 1. CONFIGURATION
# # ==========================================
# BASE_MODEL = "Qwen/Qwen2.5-1.5B-Instruct"

# DATASET_PATHS = [
#     "/content/drive/MyDrive/llmFinal/datasets/airport_ticket.csv",
#     "/content/drive/MyDrive/llmFinal/datasets/fastfood.csv",
#     "/content/drive/MyDrive/llmFinal/datasets/finance.csv",
#     "/content/drive/MyDrive/llmFinal/datasets/media.csv",
#     "/content/drive/MyDrive/llmFinal/datasets/software.csv",
# ]

# TAVILY_API_KEY = "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"
# os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY

# # ==========================================
# # 2. LOAD MODELS
# # ==========================================
# print("Loading models...")

# base_model = AutoModelForCausalLM.from_pretrained(
#     BASE_MODEL,
#     torch_dtype=torch.float16,
#     device_map="auto"
# )

# if os.path.exists(FINETUNED_PATH):
#     try:
#         from peft import PeftConfig

#         # Try to load with compatibility settings
#         config = PeftConfig.from_pretrained(FINETUNED_PATH)
#         model = PeftModel.from_pretrained(
#             base_model,
#             FINETUNED_PATH,
#             is_trainable=False,
#             low_cpu_mem_usage=True
#         )
#         print("✅ Fine-tuned adapters loaded!")
#     except Exception as e:
#         print(f"Failed to load adapters: {e}")
#         print("Using base model only.")
#         model = base_model
# else:
#     print("Path not found. Loading base model only.")
#     model = base_model

# tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# pipe = pipeline(
#     "text-generation",
#     model=model,
#     tokenizer=tokenizer,
#     max_new_tokens=150,
#     temperature=0.7,
#     repetition_penalty=1.2,
#     return_full_text=False
# )

# generator_llm = HuggingFacePipeline(pipeline=pipe)
# similarity_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

# # ==========================================
# # 3. BUILD KNOWLEDGE BASE
# # ==========================================
# print("Processing CSVs...")

# def load_conversations(csv_paths, max_per_dataset=200):
#     all_docs = []
#     for csv_path in csv_paths:
#         if not os.path.exists(csv_path):
#             continue
#         try:
#             df = pd.read_csv(csv_path)
#             if 'utterance' not in df.columns: continue

#             df = df.dropna(subset=['utterance'])
#             df['utterance'] = df['utterance'].astype(str).str.strip()

#             count = 0
#             for conv_id, group in df.groupby('conversationId'):
#                 if count >= max_per_dataset: break

#                 text = ""
#                 for _, row in group.iterrows():
#                     role = "Customer" if row.get('authorRole') == 'customer' else "Agent"
#                     text += f"{role}: {row['utterance']}\n"

#                 all_docs.append(Document(page_content=text, metadata={"source": csv_path}))
#                 count += 1
#             print(f"Loaded {count} convs from {os.path.basename(csv_path)}")
#         except:
#             continue
#     return all_docs

# docs = load_conversations(DATASET_PATHS)

# if not docs:
#     print("No documents found! Check your Drive paths.")
# else:
#     text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
#     split_docs = text_splitter.split_documents(docs)

#     vectorstore = Chroma.from_documents(
#         documents=split_docs,
#         collection_name="colab_kb_fixed",
#         embedding=HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
#     )
#     retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
#     print("Knowledge Base Ready!")

# # ==========================================
# # 4. DEFINE AGENT
# # ==========================================

# class GraphState(TypedDict):
#     question: str
#     generation: str
#     web_search: str
#     documents: List[str]
#     urgency: str      # INPUT from external agent
#     emotion: str      # INPUT from external agent

# def retrieve(state):
#     """Retrieve relevant documents"""
#     print(f"\n--- RETRIEVING: {state['question']} ---")
#     docs = retriever.invoke(state["question"])
#     return {**state, "documents": docs}

# def grade_documents(state):
#     """Grade document relevance and decide if web search is needed"""
#     question = state["question"]
#     documents = state["documents"]

#     # Check for real-time queries
#     if any(x in question.lower() for x in ["weather", "today", "now", "price", "stock"]):
#         print("  Real-time query -> WEB SEARCH")
#         return {**state, "web_search": "Yes"}

#     # No documents found
#     if not documents:
#         return {**state, "documents": [], "web_search": "Yes"}

#     # Calculate similarity
#     q_emb = similarity_model.encode(question, convert_to_tensor=True)
#     max_sim = 0
#     for doc in documents:
#         d_emb = similarity_model.encode(doc.page_content[:400], convert_to_tensor=True)
#         max_sim = max(max_sim, util.cos_sim(q_emb, d_emb).item())

#     print(f" Similarity Score: {max_sim:.2f}")

#     if max_sim < 0.4:
#         print("  Low Relevance -> WEB SEARCH")
#         return {**state, "web_search": "Yes"}

#     print("  High Relevance -> GENERATE")
#     return {**state, "web_search": "No"}

# def web_search(state):
#     """Perform web search for additional context"""
#     print("--- WEB SEARCH ---")
#     try:
#         results = TavilySearchResults(k=3).invoke({"query": state["question"]})
#         web_text = "\n".join([r["content"] for r in results])
#         documents = state["documents"] + [Document(page_content=web_text)]
#         print("  Web results found")
#         return {**state, "documents": documents}
#     except Exception as e:
#         print(f"Web search failed: {e}")
#         return state

# def generate(state):
#     """Generate response based on context, urgency, and emotion"""
#     print("--- GENERATING ---")
#     question = state["question"]
#     documents = state["documents"]
#     urgency = state.get("urgency", "medium").lower()
#     emotion = state.get("emotion", "neutral").lower()

#     context = "\n\n".join([d.page_content[:500] for d in documents])
#     print(f"CONTEXT SEEN BY MODEL:\n{context[:200]}...\n(Total chars: {len(context)})")
#     print(f"Urgency: {urgency.upper()} | Emotion: {emotion.upper()}")

#     # ========== EXACT TONE MATCHING FOR ALL SCENARIOS ==========

#     # HIGH URGENCY
#     if urgency == "high" and emotion == "angry":
#         tone_instruction = "Be extremely empathetic and apologetic. Acknowledge the issue immediately and provide concrete solutions with urgency."
#     elif urgency == "high" and emotion == "frustrated":
#         tone_instruction = "Be understanding and solution-focused. Act quickly to resolve the issue and show you're prioritizing their concern."
#     elif urgency == "high" and emotion == "anxious":
#         tone_instruction = "Be reassuring and calm while acting swiftly. Provide clear steps and timeline to reduce their anxiety."
#     elif urgency == "high" and emotion == "neutral":
#         tone_instruction = "Be direct, efficient, and action-oriented. Provide immediate solutions without unnecessary details."
#     elif urgency == "high" and emotion == "satisfied":
#         tone_instruction = "Be appreciative and maintain the positive experience. Act quickly while being warm and professional."
#     elif urgency == "high" and emotion == "happy":
#         tone_instruction = "Be enthusiastic and efficient. Match their positive energy while resolving their urgent request quickly."

#     # MEDIUM URGENCY
#     elif urgency == "medium" and emotion == "angry":
#         tone_instruction = "Be patient and understanding. Acknowledge their frustration and work through the solution methodically."
#     elif urgency == "medium" and emotion == "frustrated":
#         tone_instruction = "Be empathetic and helpful. Show you understand their concern and provide clear, actionable solutions."
#     elif urgency == "medium" and emotion == "anxious":
#         tone_instruction = "Be supportive and reassuring. Provide clear guidance and communicate each step to ease their worry."
#     elif urgency == "medium" and emotion == "neutral":
#         tone_instruction = "Be friendly, helpful, and professional. Provide thorough assistance with a balanced approach."
#     elif urgency == "medium" and emotion == "satisfied":
#         tone_instruction = "Be warm and attentive. Maintain their satisfaction while helping them with their request."
#     elif urgency == "medium" and emotion == "happy":
#         tone_instruction = "Be friendly and engaging. Match their positive mood while being helpful and thorough."

#     # LOW URGENCY
#     elif urgency == "low" and emotion == "angry":
#         tone_instruction = "Be calm, patient, and thorough. Take time to fully explain and address their concerns."
#     elif urgency == "low" and emotion == "frustrated":
#         tone_instruction = "Be understanding and patient. Provide detailed explanations and show genuine care for resolving their issue."
#     elif urgency == "low" and emotion == "anxious":
#         tone_instruction = "Be gentle, reassuring, and thorough. Take time to address their concerns and provide comprehensive support."
#     elif urgency == "low" and emotion == "neutral":
#         tone_instruction = "Be professional, informative, and helpful. Provide clear and complete information."
#     elif urgency == "low" and emotion == "satisfied":
#         tone_instruction = "Be friendly and informative. Maintain the positive relationship while helping them effectively."
#     elif urgency == "low" and emotion == "happy":
#         tone_instruction = "Be warm, friendly, and conversational. Enjoy the positive interaction while being helpful."

#     else:
#         tone_instruction = "Be helpful and professional."

#     # ========== END TONE MATCHING ==========

#     rag_system_message = f"""CRITICAL INSTRUCTIONS - FOLLOW EXACTLY:
# 1. You are a GENERAL helpful assistant, NOT a ticket agent
# 2. NEVER mention company names, websites, or booking services
# 3. Customer Urgency: {urgency.upper()}
# 4. Customer Emotion: {emotion.upper()}
# 5. Tone: {tone_instruction}

# ANSWER THE QUESTION using ONLY the Context provided below.
# If the Context has the answer, use it.
# If the Context doesn't have the answer, say "I don't have that information."
# NEVER make up information. NEVER mention companies or websites.
# Maximum 2-3 sentences."""

#     prompt = f"""<|im_start|>system
# {rag_system_message}<|im_end|>
# <|im_start|>user
# Context:
# {context}

# Question: {question}<|im_end|>
# <|im_start|>assistant
# """

#     result = pipe(prompt, max_new_tokens=150, temperature=0.5)[0]['generated_text']

#     if "<|im_start|>assistant" in result:
#         result = result.split("<|im_start|>assistant")[-1].strip()

#     return {**state, "generation": result}

# def route(state):
#     """Route to web search or generate based on document grading"""
#     return "web_search" if state["web_search"] == "Yes" else "generate"

# # ==========================================
# # 5. BUILD WORKFLOW
# # ==========================================

# workflow = StateGraph(GraphState)

# workflow.add_node("retrieve", retrieve)
# workflow.add_node("grade_documents", grade_documents)
# workflow.add_node("web_search", web_search)
# workflow.add_node("generate", generate)

# workflow.set_entry_point("retrieve")
# workflow.add_edge("retrieve", "grade_documents")
# workflow.add_conditional_edges("grade_documents", route, {"web_search": "web_search", "generate": "generate"})
# workflow.add_edge("web_search", "generate")
# workflow.add_edge("generate", END)

# app = workflow.compile()

# # ==========================================
# # 6. TEST
# # ==========================================
# print("\n" + "="*50)
# print("TEST 1: High Urgency + Angry")
# inputs = {
#     "question": "I want to book a flight to Chicago",
#     "urgency": "high",
#     "emotion": "angry"
# }
# for output in app.stream(inputs):
#     pass
# print(f"\n>>> ANSWER: {output['generate']['generation']}")

# print("\n" + "="*50)
# print("TEST 2: Medium Urgency + Neutral")
# inputs = {
#     "question": "What is the weather in Tokyo right now?",
#     "urgency": "medium",
#     "emotion": "neutral"
# }
# for output in app.stream(inputs):
#     pass
# print(f"\n>>> ANSWER: {output['generate']['generation']}")

Loading models...
Path not found. Loading base model only.


Device set to use cuda:0


Processing CSVs...
Loaded 200 convs from airport_ticket.csv
Loaded 200 convs from fastfood.csv
Loaded 200 convs from finance.csv
Loaded 200 convs from media.csv
Loaded 200 convs from software.csv
Knowledge Base Ready!

TEST 1: High Urgency + Angry

--- RETRIEVING: I want to book a flight to Chicago ---
 Similarity Score: 0.53
  High Relevance -> GENERATE
--- GENERATING ---
CONTEXT SEEN BY MODEL:
Customer: hi
Agent: Good morning! You've reached Alliance Airways Customer service, how may I help you?
Customer: "Book" a plane ticket
Agent: Sure! I'll help you with that, could you please let me kn...
(Total chars: 1504)
Urgency: HIGH | Emotion: ANGRY

>>> ANSWER: I'm sorry for any inconvenience caused. To assist you better, do you need assistance in finding available flights from your current location (Rochester) on August 29? If so, we can look into options including different times of day such as mornings if that's what you're interested in. Please confirm which time slot is preferred by 

In [7]:
# ==========================================
# 6. TEST ALL 18 SCENARIOS
# ==========================================

test_cases = [
    # HIGH URGENCY
    ("medium", "anxious", "My flight leaves in 2 hours and I still haven't received my booking confirmation!"),
    ("high", "frustrated", "I've been trying to track my package for hours. I need this today!"),
    ("medium", "anxious", "I'm really worried about my order. It was supposed to arrive yesterday."),
    ("high", "anxious", "I need to change my reservation immediately. What are my options?"),
    ("high", "neutral", "Thank you for the service! I need to add one more item urgently."),
    ("medium", "happy", "This is amazing! Can you help me order three more right away?"),

    # MEDIUM URGENCY
    ("medium", "frustrated", "I'm not happy with the service. The product description was misleading."),
    ("medium", "frustrated", "I've contacted support twice about this. Can someone help?"),
    ("medium", "anxious", "I'm concerned about the payment process. Is my information secure?"),
    ("medium", "neutral", "Can you explain how the refund policy works?"),
    ("low", "happy", "I'm happy with my purchase. Tell me about your loyalty program."),
    ("low", "happy", "I'm enjoying my experience! What other products do you recommend?"),

    # LOW URGENCY
    ("medium", "frustrated", "I'm disappointed with this. I want a full explanation of what went wrong."),
    ("medium", "anxious", "This is confusing. Why is the process so complicated?"),
    ("medium", "anxious", "I'm not sure if I made the right choice. Can you explain the differences?"),
    ("medium", "neutral", "What are the technical specifications of this product?"),
    ("low", "happy", "Everything's been great. I'm curious about your sustainability practices."),
    ("low", "happy", "I love your products! Tell me about the story behind your company.")
]

print("\n" + "="*70)
print("TESTING 18 SCENARIOS: 3 URGENCY × 6 EMOTIONS")
print("="*70)

success_count = 0
web_search_count = 0

for i, (urgency, emotion, query) in enumerate(test_cases, 1):
    print(f"\n{'='*70}")
    print(f"TEST {i}/18: {urgency.upper()} + {emotion.upper()}")
    print(f"{'='*70}")
    print(f"Query: {query}")
    print("-"*70)

    inputs = {
        "question": query,
        "urgency": urgency,
        "emotion": emotion
    }

    try:
        for output in app.stream(inputs):
            pass

        response = output['generate']['generation']
        web_search = output['generate'].get('web_search', 'No')

        print(f"Web Search: {web_search}")
        print(f"\n>>> RESPONSE:\n{response}\n")

        success_count += 1
        if web_search == "Yes":
            web_search_count += 1

    except Exception as e:
        print(f"❌ ERROR: {e}\n")

print("\n" + "="*70)
print("SUMMARY")
print("="*70)
print(f"Successful: {success_count}/18")
print(f"Web Search Triggered: {web_search_count}/18")
print("="*70)


TESTING 18 SCENARIOS: 3 URGENCY × 6 EMOTIONS

TEST 1/18: MEDIUM + ANXIOUS
Query: My flight leaves in 2 hours and I still haven't received my booking confirmation!
----------------------------------------------------------------------

--- RETRIEVING: My flight leaves in 2 hours and I still haven't received my booking confirmation! ---
 Similarity Score: 0.65
  High Relevance -> GENERATE
--- GENERATING ---
CONTEXT SEEN BY MODEL:
Agent: Oh! That's okay! I'll change that. However, As I can check here both the numbers given to me are incorrect, Your booking confirmation number should be 6 characters (mix of letters and numbers)....
(Total chars: 1228)
Urgency: MEDIUM | Emotion: ANXIOUS
Web Search: No

>>> RESPONSE:
The customer booked with an airline but hasn’t yet received their boarding pass due to receiving two different confirmation numbers instead of one correct one. The staff is working hard to resolve this issue by contacting the relevant department for assistance; they will ensure